# Examples and applications of Kolmogorov-Smirnov tests, Permutation tests, and Monte Carlo resampling techniques

### Kolmogorov-Smirnov test

The KS test:
- is only valid for continuous distributions
- tends to be more sensitive near the center of the distribution than at the tails
- requires a fully specified distribution. That is, if location, scale, and shape parameters are estimated from the data, the critical region of the K-S test is no longer valid. 

The Kolmogorov-Smirnov test can be used to answer the following types of questions:

- Are the data from a normal distribution?
- Are the data from a log-normal distribution?
- Are the data from a Weibull distribution?
- Are the data from an exponential distribution?
- Are the data from a logistic distribution?

Let's take a crack at some quick examples to famliarize ourselves with the KS test.

###  1 sample Kolmogorov-Smirnov test

The 1 sample Kolmogorov-Smirnov test compares the distribution G(x) of an observed random variable against a given distribution F(x). 

Under the null hypothesis the two distributions are identical, G(x)=F(x). 

The alternative hypothesis can be either ‘two-sided’ (default), ‘less’ or ‘greater’.


In [3]:
from scipy import stats
import numpy as np

In [4]:
x = np.linspace(-15, 15, 9)
stats.kstest(x, 'norm')

KstestResult(statistic=0.4443560271592436, pvalue=0.038850142705171065)

In [5]:
np.random.seed(987654321) # set random seed to get the same result
stats.kstest('norm', False, N=100)

KstestResult(statistic=0.058352892479417884, pvalue=0.8853119094415126)

The above lines are equivalent to:

In [6]:
np.random.seed(987654321)
stats.kstest(stats.norm.rvs(size=100), 'norm')

KstestResult(statistic=0.058352892479417884, pvalue=0.8853119094415126)

#### Test against one-sided alternative hypothesis

Shift distribution to larger values, so that cdf_dgp(x) < norm.cdf(x):

In [7]:
np.random.seed(987654321)
x = stats.norm.rvs(loc=0.2, size=100)
stats.kstest(x,'norm', alternative = 'less')

KstestResult(statistic=0.12464329735846891, pvalue=0.04098916407764175)

Reject equal distribution against alternative hypothesis: less

In [8]:
stats.kstest(x,'norm', alternative = 'greater')

KstestResult(statistic=0.007211523321631108, pvalue=0.985311585903964)

Don’t reject equal distribution against alternative hypothesis: greater

In [9]:
stats.kstest(x,'norm', mode='asymp')

KstestResult(statistic=0.12464329735846891, pvalue=0.08944488871182082)

#### Testing t distributed random variables against normal distribution

With 100 degrees of freedom the t distribution looks close to the normal distribution, and the K-S test does not reject the hypothesis that the sample came from the normal distribution:

In [10]:
np.random.seed(987654321)
stats.kstest(stats.t.rvs(100,size=100),'norm')

KstestResult(statistic=0.07201892916547126, pvalue=0.6763006286247917)

With 3 degrees of freedom the t distribution looks sufficiently different from the normal distribution, that we can reject the hypothesis that the sample came from the normal distribution at the 10% level:

In [11]:
np.random.seed(987654321)
stats.kstest(stats.t.rvs(3,size=100),'norm')

KstestResult(statistic=0.131016895759829, pvalue=0.058826222555312224)

### 2 sample KS test

In [12]:
np.random.seed(12345678)  #fix random seed to get the same result
n1 = 200  # size of first sample
n2 = 300  # size of second sample

For a different distribution, we can reject the null hypothesis since the pvalue is below 1%:

In [13]:
rvs1 = stats.norm.rvs(size=n1, loc=0., scale=1)
rvs2 = stats.norm.rvs(size=n2, loc=0.5, scale=1.5)
stats.ks_2samp(rvs1, rvs2)

Ks_2sampResult(statistic=0.20833333333333337, pvalue=4.667497551580699e-05)

For a slightly different distribution, we cannot reject the null hypothesis at a 10% or lower alpha since the p-value at 0.144 is higher than 10%

In [15]:
rvs3 = stats.norm.rvs(size=n2, loc=0.01, scale=1.0)
stats.ks_2samp(rvs1, rvs3)

Ks_2sampResult(statistic=0.10333333333333333, pvalue=0.14498781825751686)

For an identical distribution, we cannot reject the null hypothesis since the p-value is high, 41%:

In [17]:
rvs4 = stats.norm.rvs(size=n2, loc=0.0, scale=1.0)
stats.ks_2samp(rvs1, rvs4)

Ks_2sampResult(statistic=0.07999999999999996, pvalue=0.4112694972985972)

KS tests are helpful to use when you are interested in using a particular statistical test that demands/assumes a particular distribution. They can be used to verify your results in this way.

### Resampling techniques

A method that consists of drawing repeated samples from the original data samples.

#### Why do we want to do this?

There are three instances when we would want to use resampling:

- When we want to estimate the precision of sample statistics (medians, variances, percentiles)
- When exchanging labels on data points when performing significance tests
- When we want to validate models by using random subsets

#### How do we resample our data?

- Randomization (Permutation) Test
- Monte Carlo Simulation
- Bootstrapping



#### Permutation testing

Permutation tests are a group of nonparametric statistics. Here we use a permutation test to test the null hypothesis that two different groups come from the same distribution.

In a permutation test, you are essentially just shuffling your data in order to get a randomized selection of your data, so you can test things like whether or not two sets of data come from the same distribution.

In [52]:
z = np.array([94,197,16,38,99,141,23])
y = np.array([52,104,146,10,51,30,40,27,46])

In [55]:
theta_hat = z.mean() - y.mean()
theta_hat

30.63492063492064

In [68]:
#create a function to run your permutation

def run_permutation_test(pooled,sizeZ,sizeY,delta):
    #randomize all the pooled data from arrays y and z
    np.random.shuffle(pooled)
    #grab data the length of z
    starZ = pooled[:sizeZ]
    #grab the rest of the data the length of y
    starY = pooled[-sizeY:]
    #find the difference in the mean
    return starZ.mean() - starY.mean()

In [72]:
pooled = np.hstack([z,y])
delta = z.mean() - y.mean()
numSamples = 10000
estimates = np.array(list(map(lambda x: run_permutation_test(pooled,z.size,y.size,delta),range(numSamples))))
diffCount = len(np.where(estimates < delta)[0])
hat_sig_perm = 1.0 - (float(diffCount)/float(numSamples))
hat_sig_perm

0.14459999999999995

In this case, we fail to reject the null hypothesis.

Here is another tool that I find useful sometimes. It is a library called mlxtend.

There are some very helpful functions in this package. If you do not want to use this tool, that is also okay! The results are a bit different sometimes, but that is because the tools are programmed a bit differently. I have provided the link to the github repo with all of this code at the bottom of the notebook if you're interested in investigating that further.

In [ ]:
#Install mlxtend (if you want)

!pip install mlxtend

In [74]:
from mlxtend.evaluate import permutation_test

p_value = permutation_test(z, y,
                           num_rounds=10000,
                           seed=0)
print(p_value)

0.27604895104895105


We fail to reject the null hypothesis.

### Monte Carlo Simulation

#### What is Monte Carlo simulation?

Monte Carlo simulation is a method used to model probabilistic systems and establish the odds for a variety of outcomes.

For example:

![https://www.greatlakespondscapes.com/wp-content/uploads/GreatLakesPondscapes-27_480x320_acf_cropped.jpg](https://www.greatlakespondscapes.com/wp-content/uploads/GreatLakesPondscapes-27_480x320_acf_cropped.jpg)

You have a rectangular yard that is 10 by 15 feet. There is a pond in the middle of irregular shape and you want to know the area of the pond. 

You start throwing small stones toward the yard randomly and record the frequency of stone landing on water versus dry land. The portion of stones landing on water multiplied by the entire area of the backyard is the answer to the original question. 

For example, if you threw 100 stones and 43 lands on water, then your estimation would be 10 times 15 times 0.43. This is essentially the essence of Monte Carlo: evaluating and averaging randomly drawn samples.


#### Generalized steps to Monte Carlo simulation

1. Construct a simulated “universe” of some other randomizing mechanism whose composition is similar to the universe whose behavior we wish to describe and investigate, like dice or cards or coins. The term “universe” refers to the system that is relevant for a single simple event. In this step, you will do the following:
    - Decide which symbols will stand for the elements of the universe you will simulate.
    - Determine whether the sampling will be with or without replacement. (This can be ambiguous in a complex modeling situation.) Note: if trials are independent you will sample with replacement.
2. Specify the procedure that produces a pseudo-sample  which simulates the real-life sample in which we are interested. That is, specify the procedural rules by which the sample is drawn from the simulated universe. These rules must correspond to the behavior of the real universe in which you are interested. To put it another way, the simulation procedure must produce simple experimental events with the same probabilities that the simple events have in the real world.
3. If several simple events must be combined into a composite event, and if the composite event was not described in the procedure in step 2, describe it now.
4. Calculate the probability of interest from the tabulation of outcomes of the resampling trials.


Let's try applying these generalized rules to some specific situations.

If the birth rate ratio of boys to girls is 51:49, what is the probability of having two children who are both girls. 



In [20]:
import random

In [30]:
#Create a sample universe. Here we have just created a list of 100 babies 
#that are boys or girls. This event can, however, be represented by the same mechanic as
#flipping a coin, too.

babies = []
for i in range(0,49):
    babies += ['g']
for i in range(0,51):
    babies += ['b']


Answer = 0
Simulations = 1000000


In [31]:
#run Monte Carlo simulation
for i in range(0,Simulations):
    girls = random.sample(babies, 2)
    if (girls[0]== "g" and girls[1]=="g"):
        Answer += 1

In [32]:
#get resulting probablity
Result = Answer/Simulations
print(Result)

0.237327


Do note, if you run the above cells again, you will yield a slightly different result, becuase there is a note of randomness to this technique.

#### Bootstrap resampling

Bootstrap is a special case of the Monte Carlo simulation. The differences between the bootstrap and Monte Carlo methods are:
- Monte Carlo simulation is based on a random number and a a priori distribution form of data
- Bootstrap resamples with replacement on the real data.

Samples are drawn from the dataset with replacement (allowing the same sample to appear more than once in the sample), where those instances not drawn into the data sample may be used for the test set.

The most widely used resampling method, it estimates the sampling distribution of an estimator by sampling with replacement from the original estimate, most often with the purpose of deriving robust estimates of standard errors and confidence intervals of a population parameter.

The steps for bootstrap resampling are the following, generally:
- Start with a sample from population with sample size n.
- Draw a sample from the original sample data with replacement of size n, where each re-sampled sample is called a Bootstrap Sample.
- Evaluate the statistics of the metric you are estimating for each bootstrap sample.
- Construct a sampling distribution with these B Bootstrap statistics and use it to make further statistical inference, such as:
  - Estimating the standard error of the statistics of your metric.
  - Obtaining a Confidence Interval for your metric.


Imagine that you want to summarize how many times a day people are on their phone on the 4th floor, where the total number of people is 100 (for sake of ease). Over the next few days, you ask 30 others on the 4th floor how many times they are on their phone in a day. From these responses,  you calculated the mean of these 30 responses and got an estimate for phone calls  is 15 times a day.

In [38]:
# construct a population pickups for our lab
np.random.seed(42)
calls = np.random.randint(0,30 , size=100)
calls

# draw a sample from population
sample = np.random.choice(calls, size=30)
sample

array([27, 17,  1, 10, 21, 14, 23,  1, 11,  7, 17, 26,  3,  7,  2, 19, 23,
       29,  1,  8,  9, 27, 23, 14, 27,  1, 20,  6,  3,  6])

In [39]:
calls.mean()

15.63

In [40]:
calls.std()

8.988498206040873

In [41]:
# bootstrap for mean
boot_means = []
for _ in range(10000):
    bootsample = np.random.choice(sample,size=30, replace=True)
    boot_means.append(bootsample.mean())

In [42]:
# simulated mean of mean
bootmean = np.mean(boot_means)

In [43]:
# simulated standard deviation of mean
bootmean_std = np.std(boot_means)

In [45]:
# simulated mean VS true mean
(calls.mean(), bootmean)

(15.63, 13.437966666666668)

In [46]:
# the theorical standard error and simulated standard error
(calls.std()/(30 ** 0.5), bootmean_std)

(1.6410677418477682, 1.6943836896576223)

## Additional resources

- https://towardsdatascience.com/monte-carlo-simulations-with-python-part-1-f5627b7d60b0
- https://towardsdatascience.com/the-house-always-wins-monte-carlo-simulation-eb82787da2a3

- https://github.com/rasbt/mlxtend Github repo for mlxtend. This package has very useful functions within it.